<a href="https://colab.research.google.com/github/dezlorator1/DSS/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg  
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten
from PIL import Image
from google.colab import drive
from google.colab import files

ПОДГОТОВ ОЧКА КАРТИНОК


In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img = mpimg.imread('/content/drive/MyDrive/img/CutMoscow.jpg')

plt.figure()
plt.imshow(img)

In [ ]:
chopsize = 50
infile = r"/content/drive/MyDrive/img/CutMoscow.jpg"
outfile = r"/content/drive/MyDrive/img/cut_img_color/"
img = Image.open(infile)
width, height = img.size
count = 0 ;
# Save Chops of original image
for x0 in range(0, width, chopsize):
   for y0 in range(0, height, chopsize):
      box = (x0, y0,
             x0+chopsize if x0+chopsize <  width else  width - 1,
             y0+chopsize if y0+chopsize < height else height - 1)
      #print('%s %s' % (infile, box))
      img.crop(box).save(outfile + f'{count}' + 'c.jpg', "JPEG")
      count = count + 1

In [ ]:
files.upload()

ЗАГОТОВКА МАССИВОВ ДАННЫХ

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_black_dir = r"/content/drive/MyDrive/img_black"
df_black = ImageDataGenerator(rescale=1. / 255)

In [ ]:
df_black_gen = df_black.flow_from_directory(
    train_black_dir,
    target_size=(50, 50),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

In [ ]:
x_black, _ = df_black_gen.next()

In [ ]:
train_color_dir = r"/content/drive/MyDrive/img_color/"
df_color = ImageDataGenerator(rescale=1. / 255)

In [ ]:
df_color_gen = df_color.flow_from_directory(
    train_color_dir,
    target_size=(50, 50),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32,
    shuffle=False
)

Found 960 images belonging to 1 classes.


In [ ]:
y_color, _ = df_color_gen.next()

НА ЭТО ГОВНО СМОТРЕТЬ НЕ НАДО

А ПРИЙДЕТСЯ...

In [ ]:
cut_black = np.array([])
#black = np.array([])
for i in range(2):
  img = mpimg.imread('/content/drive/MyDrive/img/cut_img_black/' + f'{i}' + '.jpg')
  #black = np.append(black, img)
  cut_black = np.append(cut_black, img)
  print(cut_black)
  #cut_black.append(img)

In [ ]:
cut_color = np.array([])
for i in range(960):
    img = mpimg.imread('/content/drive/MyDrive/img/cut_img_color/' + f'{i}' + 'c.jpg')
    cut_color = np.append(cut_color, img)
    #cut_color.append(img)

In [ ]:
print(cut_black[0])

In [ ]:
plt.figure()
plt.imshow(cut_black[0])
#plt.figure()
#plt.imshow(cut_color[0])


In [ ]:
for i in range(960):
  cut_black[i] = cut_black[i] / 255
  cut_color[i] = cut_color[i] / 255

НЕЙРОНКА


In [ ]:
model = keras.Sequential([
    Flatten(input_shape=(50, 50, 3)),
    Dense(2000, activation='relu'),
    Dense(500, activation='relu'),
    Dense(7500,input_shape=(50, 50, 3), activation='softmax')
])

In [ ]:
print(model.summary()) 

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
model.fit(x_black, y_color, batch_size=32, epochs=5, validation_split=0.15)